In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import re
import numpy as np
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [31]:
df = pd.read_csv('./DataSet/reddit_PSE_ISR_simple.csv')

In [32]:
for i in range(10):
    print(df.self_text[i])

No mas Hamas.
Israel=100% colonizer.  Indigenous Palestinian Jews, Samaritans and Christians all face the same genocide.   Yes, Jews are indigenous to Palestine and those Jews are killed by colonizing Jews.
Yeah so this is the exact thing their ancestors were told. It was what my Pontian great grandparents were told. It was what the assyrians and Greek communities were told. 

Maybe in context the forcefulness of the threat is a bit stronger. Like when the people who helped genocide you once before start saying the same things it has added weight
There are 35-45k estimated Hamas terrorist operating in Gaza. This beyond just using special forces and surgical strikes. Israel needs to continue what they are doing sending in ground troops to cut Gaza apart piece by piece. Isolating and killing pockets of terrorists as they move deeper into Gaza.
People keep going on about the 1967 borders. If they wanted the 1967 borders they should have accepted them then instead of launching the 6 Days W

In [27]:
def keep_main_text(text):
    text = str(text)
    # 使用正则表达式匹配英文字符、标点符号和空格  
    pattern = r'[ !\"a-zA-Z.,?;:\']'  
    # pattern = r'[a-zA-Z.,!?;: \n\t]'
    matches = re.findall(pattern, text)  
      
    # 输出结果  
    return ''.join(matches)

In [28]:
df['self_text'] = df['self_text'].map(keep_main_text)

In [30]:
for i in range(10):
    print(df.self_text[i])

No mas Hamas.
Israel colonizer.  Indigenous Palestinian Jews, Samaritans and Christians all face the same genocide.   Yes, Jews are indigenous to Palestine and those Jews are killed by colonizing Jews.
Yeah so this is the exact thing their ancestors were told. It was what my Pontian great grandparents were told. It was what the assyrians and Greek communities were told. Maybe in context the forcefulness of the threat is a bit stronger. Like when the people who helped genocide you once before start saying the same things it has added weight
There are k estimated Hamas terrorist operating in Gaza. This beyond just using special forces and surgical strikes. Israel needs to continue what they are doing sending in ground troops to cut Gaza apart piece by piece. Isolating and killing pockets of terrorists as they move deeper into Gaza.
People keep going on about the  borders. If they wanted the  borders they should have accepted them then instead of launching the  Days War. If you reject a t

In [34]:
def clean(s):
    s = str(s)
    s = s.replace('/',' or ').replace('&',' and ')
    #设置缩略词的正则表达式
    pat_is = re.compile("(it|he|she|that|this|there|here)(\'s)",re.I)
    pat_s = re.compile("(?<=[a-zA-Z])\'s")
    pat_s2 = re.compile("(?<=s)\'s?")
    pat_not = re.compile("(?<=[a-zA-Z])\'t")
    pat_would = re.compile("(?<=[a-zA-Z])\'d")
    pat_will = re.compile("(?<=[a-zA-Z])\'ll")
    pat_am = re.compile("(?<=[I|i])\'m")
    pat_are = re.compile("(?<=[a-zA-Z])\'re")
    pat_ve = re.compile("(?<=[a-zA-Z])\'ve")
    #使用正则表达式进行展开
    new_text = pat_is.sub(r"\1 is", s)
    new_text = pat_s.sub("",new_text)
    new_text = pat_s2.sub("",new_text)
    new_text = pat_not.sub(" not",new_text)
    new_text = pat_would.sub(" would",new_text)
    new_text = pat_will.sub(" will",new_text)
    new_text = pat_am.sub(" am",new_text)
    new_text = pat_are.sub(" are",new_text)
    new_text = pat_ve.sub(" have",new_text)
    new_text = new_text.replace('\'',' ')
    # print(new_text)
    return new_text

In [41]:
df['self_text'] = df['self_text'].map(clean)

In [42]:
for i in range(10):
    print(df.self_text[i])

No mas Hamas.
Israel colonizer.  Indigenous Palestinian Jews, Samaritans and Christians all face the same genocide.   Yes, Jews are indigenous to Palestine and those Jews are killed by colonizing Jews.
Yeah so this is the exact thing their ancestors were told. It was what my Pontian great grandparents were told. It was what the assyrians and Greek communities were told. Maybe in context the forcefulness of the threat is a bit stronger. Like when the people who helped genocide you once before start saying the same things it has added weight
There are k estimated Hamas terrorist operating in Gaza. This beyond just using special forces and surgical strikes. Israel needs to continue what they are doing sending in ground troops to cut Gaza apart piece by piece. Isolating and killing pockets of terrorists as they move deeper into Gaza.
People keep going on about the  borders. If they wanted the  borders they should have accepted them then instead of launching the  Days War. If you reject a t

In [44]:
df.to_csv('./DataSet/reddit.csv', index = False)

In [55]:
df = pd.read_csv('./DataSet/reddit.csv')
df = df.dropna(how='any', subset=['self_text']).reset_index(drop=True)
df

,created_time,self_text
0,2023-12-02 23:26:34,No mas Hamas.
1,2023-12-02 23:26:17,Israel colonizer. Indigenous Palestinian Jews...
2,2023-12-02 23:26:17,Yeah so this is the exact thing their ancestor...
3,2023-12-02 23:26:15,There are k estimated Hamas terrorist operatin...
4,2023-12-02 23:26:12,People keep going on about the borders. If th...
...,...,...
333806,2023-09-02 13:57:37,Something similar happened in Sweden recently.
333807,2023-09-02 13:33:15,They should leave the anger and hatred back wh...
333808,2023-09-02 13:24:53,Something similar happened in Germany recently...
333809,2023-09-02 12:17:34,Who are the yellow shirts? The referees?


In [56]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string

# 获取单词的词性
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

stopword = stopwords.words('english')
def spilt(text):
    text = text.lower()
    tokens = word_tokenize(text)  # 分词
    tagged_sent = pos_tag(tokens)     # 获取单词词性
    
    wnl = WordNetLemmatizer()
    lemmas_sent = []
    for tag in tagged_sent:
        wordnet_pos = get_wordnet_pos(tag[1])
        if wordnet_pos == wordnet.NOUN:
            c = wnl.lemmatize(tag[0], pos=wordnet_pos) # 词形还原
            if c not in stopword and c not in string.punctuation:
                lemmas_sent.append(c)
    
    return " ".join(lemmas_sent)

In [57]:
df['self_text'] = df['self_text'].map(spilt)

In [61]:
for i in range(20):
    print(df.self_text[i])
df.info()

hamas
colonizer jew samaritan christian genocide jew jew jew
thing ancestor grandparent assyrian community context forcefulness threat people start thing weight
k operating gaza force strike israel ground troop piece piece isolating pocket terrorist gaza
people border border day war treaty war extermination territory peace border launch war engage terrorism year deal
summer child number people sane mind
arab invader state solution accepts reject war sound history
feel
uk guy lot stuff ww famine course people raf bombing cities.also weird john government
condemnation condemnation
hamas
take people society suffers lot racism okay civilian half country earth people israel lebanon judaism forbids activity sex marriage israel fact palestinian racism racist arab video man ramallah respect arab racist black black video guy ramallah http youtu.bectyeozxo issue palestine israel issue right dignity land living people gender rhetoric
school city gaza u one country anymore refugee
matter stake itl

In [62]:
df = df.replace('', np.nan)
df = df.dropna(how='any', subset=['self_text']).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321873 entries, 0 to 321872
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   created_time  321873 non-null  object
 1   self_text     321873 non-null  object
dtypes: object(2)
memory usage: 4.9+ MB


In [63]:
df.to_csv('./DataSet/reddit_clean.csv', index=False)

In [68]:
df1 = df[(df['created_time'] >= '2023-10-07')]

In [66]:
df1.to_csv('./ENdata/gt1007.csv', index=False) # 时间大于10月07日的评论

In [69]:
df1 = df1.rename(columns={'created_time': 'time', 'self_text': 'text'}) 

In [71]:
df1.to_csv('./ENdata/gt1007.csv', index=False)

In [2]:
df = pd.read_csv('./ENdata/gt1007.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319365 entries, 0 to 319364
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   time    319365 non-null  object
 1   text    319365 non-null  object
dtypes: object(2)
memory usage: 4.9+ MB


In [3]:
df_1 = df[(df['time'] >= '2023-10-01') & (df['time'] <= '2023-11-01')]
df_1.info()
df_1.to_csv('./ENdata/reddit_10.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 22905 entries, 296460 to 319364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   time    22905 non-null  object
 1   text    22905 non-null  object
dtypes: object(2)
memory usage: 536.8+ KB


In [4]:
df_2 = df[(df['time'] >= '2023-11-01') & (df['time'] <= '2023-12-01')]
df_2.info()
df_2.to_csv('./ENdata/reddit_11.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 275408 entries, 21052 to 296459
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   time    275408 non-null  object
 1   text    275408 non-null  object
dtypes: object(2)
memory usage: 6.3+ MB


In [80]:
df_3 = df[(df['time'] >= '2023-11-04') & (df['time'] <= '2023-11-18')]
df_3.info()
df_3.to_csv('./ENdata/reddit_3.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 125290 entries, 161239 to 286528
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   time    125290 non-null  object
 1   text    125290 non-null  object
dtypes: object(2)
memory usage: 2.9+ MB


In [81]:
df_4 = df[(df['time'] >= '2023-11-18') & (df['time'] <= '2023-12-02')]
df_4.info()
df_4.to_csv('./ENdata/reddit_4.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 151412 entries, 9827 to 161238
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   time    151412 non-null  object
 1   text    151412 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


In [82]:
df_5 = df[(df['time'] >= '2023-12-02') & (df['time'] <= '2023-12-03')]
df_5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9827 entries, 0 to 9826
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   time    9827 non-null   object
 1   text    9827 non-null   object
dtypes: object(2)
memory usage: 230.3+ KB
